In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

from scipy import stats
from scipy.stats import kendalltau
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("ecommerce.csv")

In [3]:
df.columns

Index(['Order ID', 'Order Date', 'Ship Date', 'Aging', 'Ship Mode',
       'Product Category', 'Product', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping Cost', 'Order Priority', 'Customer ID',
       'Customer Name', 'Segment', 'City', 'State', 'Country', 'Region',
       'Months'],
      dtype='object')

In [4]:
df.head()

,Order ID,Order Date,Ship Date,Aging,Ship Mode,Product Category,Product,Sales,Quantity,Discount,...,Shipping Cost,Order Priority,Customer ID,Customer Name,Segment,City,State,Country,Region,Months
0,AU-2015-1,11/9/15,11/17/15,8.0,First Class,Auto & Accessories,Car Media Players,$140.00,2,0.05,...,$4.60,Medium,LS-001,Lane Daniels,Consumer,Brisbane,Queensland,Australia,Oceania,Nov
1,AU-2015-2,6/30/15,7/2/15,2.0,First Class,Auto & Accessories,Car Speakers,$211.00,3,0.03,...,$11.20,Medium,IZ-002,Alvarado Kriz,Home Office,Berlin,Berlin,Germany,Central,Jun
2,AU-2015-3,12/5/15,12/13/15,8.0,First Class,Auto & Accessories,Car Body Covers,$117.00,5,0.01,...,$3.10,Critical,EN-003,Moon Weien,Consumer,Porirua,Wellington,New Zealand,Oceania,Dec
3,AU-2015-4,5/9/15,5/16/15,7.0,First Class,Auto & Accessories,Car & Bike Care,$118.00,2,0.05,...,$2.60,High,AN-004,Sanchez Bergman,Corporate,Kabul,Kabul,Afghanistan,Central Asia,May
4,AU-2015-5,7/9/15,7/18/15,9.0,First Class,Auto & Accessories,Tyre,$250.00,1,0.04,...,$16.00,Critical,ON-005,Rowe Jackson,Corporate,Townsville,Queensland,Australia,Oceania,Jul


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order ID          51290 non-null  object 
 1   Order Date        51289 non-null  object 
 2   Ship Date         51290 non-null  object 
 3   Aging             51289 non-null  float64
 4   Ship Mode         51290 non-null  object 
 5   Product Category  51290 non-null  object 
 6   Product           51290 non-null  object 
 7   Sales             51290 non-null  object 
 8   Quantity          51289 non-null  object 
 9   Discount          51290 non-null  object 
 10  Profit            51290 non-null  object 
 11  Shipping Cost     51290 non-null  object 
 12  Order Priority    51288 non-null  object 
 13  Customer ID       51289 non-null  object 
 14  Customer Name     51290 non-null  object 
 15  Segment           51289 non-null  object 
 16  City              51290 non-null  object

In [6]:
df.columns = [column.lower() for column in df.columns]
df.columns = df.columns.str.replace(' ', '_')

In [7]:
df = df.dropna()
df = df.drop('segment', axis=1)

In [8]:
df.columns

Index(['order_id', 'order_date', 'ship_date', 'aging', 'ship_mode',
       'product_category', 'product', 'sales', 'quantity', 'discount',
       'profit', 'shipping_cost', 'order_priority', 'customer_id',
       'customer_name', 'city', 'state', 'country', 'region', 'months'],
      dtype='object')

In [9]:
df['sales'] = df['sales'].str.replace('$',"")
df['sales'] = df['sales'].str.replace('.',"")
df = df[df.sales != ("0.xf")]
df = df[df.sales != ("0xf")]
df = df[df.sales != ('0.5.26')]


In [10]:
# Menghapus dua karakter paling belakang dari setiap nilai dalam kolom
df['sales'] = df['sales'].str.slice(stop=-2)
df['sales'] = df['sales'].str.slice(stop=-1)

# Mengubah kolom 'Angka' menjadi tipe integer
df['sales'] = df['sales'].astype(int)


In [11]:
df = df[df.discount != ('xxx') ] 
df = df[df.discount != ('test') ] 

In [12]:
df['discount'] = df['discount'].astype(float)
df['discount'] = df['discount'] * 100

In [13]:
df['shipping_cost'] = df['shipping_cost'].str.replace('$',"")
df = df[df ['shipping_cost']!= ('test') ]
df['shipping_cost'] = df['shipping_cost'].astype(float)

In [14]:
df = df[df ['ship_mode']!= ('45788') ]

In [15]:
df = df[df.region != ('So3th') ] 
df = df[df.region != ('4orth') ] 

In [16]:
df['profit'] = df['profit'].str.replace('$',"")
df['profit'] = df['profit'].astype(float)
df = df[df.quantity != ("abc")] 
df['quantity'] = df['quantity'].astype(float)

In [17]:
type_float = ['aging', 'quantity', 'discount', 'profit', 'shipping_cost']

for column in type_float:
    df[column] = df[column].astype(int)

In [18]:
date_columns = ["order_date", "ship_date"]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

In [19]:
df.head().T

,0,1,2,3,4
order_id,AU-2015-1,AU-2015-2,AU-2015-3,AU-2015-4,AU-2015-5
order_date,2015-11-09 00:00:00,2015-06-30 00:00:00,2015-12-05 00:00:00,2015-05-09 00:00:00,2015-07-09 00:00:00
ship_date,2015-11-17 00:00:00,2015-07-02 00:00:00,2015-12-13 00:00:00,2015-05-16 00:00:00,2015-07-18 00:00:00
aging,8,2,8,7,9
ship_mode,First Class,First Class,First Class,First Class,First Class
product_category,Auto & Accessories,Auto & Accessories,Auto & Accessories,Auto & Accessories,Auto & Accessories
product,Car Media Players,Car Speakers,Car Body Covers,Car & Bike Care,Tyre
sales,140,211,117,118,250
quantity,2,3,5,2,1
discount,5,3,1,5,4


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51275 entries, 0 to 51289
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          51275 non-null  object        
 1   order_date        51275 non-null  datetime64[ns]
 2   ship_date         51275 non-null  datetime64[ns]
 3   aging             51275 non-null  int32         
 4   ship_mode         51275 non-null  object        
 5   product_category  51275 non-null  object        
 6   product           51275 non-null  object        
 7   sales             51275 non-null  int32         
 8   quantity          51275 non-null  int32         
 9   discount          51275 non-null  int32         
 10  profit            51275 non-null  int32         
 11  shipping_cost     51275 non-null  int32         
 12  order_priority    51275 non-null  object        
 13  customer_id       51275 non-null  object        
 14  customer_name     51275 non

In [21]:
cat_col = df.select_dtypes(include=['object']).columns.tolist()

for i in cat_col:
    print(i)
    print(df[i].value_counts())
    print('-'*30)

order_id
order_id
AU-2015-1        1
FA-2015-13688    1
FA-2015-13678    1
FA-2015-13679    1
FA-2015-13680    1
                ..
HF-2015-6904     1
HF-2015-6905     1
HF-2015-6906     1
HF-2015-6907     1
FA-2015-30775    1
Name: count, Length: 51275, dtype: int64
------------------------------
ship_mode
ship_mode
Standard Class    30775
Second Class      10309
First Class        7490
Same Day           2701
Name: count, dtype: int64
------------------------------
product_category
product_category
Fashion               30775
Home & Furniture      10309
Auto & Accessories     7490
Electronic             2701
Name: count, dtype: int64
------------------------------
product
product
Sports Wear               2827
Fossil Watch              2796
Titak watch               2795
Sneakers                  2795
Formal Shoes              2795
Running Shoes             2795
Casula Shoes              2795
Suits                     2795
T - Shirts                2794
Shirts                    2794

In [22]:
df.to_csv('ecommerce_data_cleaned1.csv', index=False)